In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [ ]:
df = data.copy()

In [ ]:
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


In [ ]:
def filter_vacancies(df, start_hour, end_hour, min_rating):
    filtered_df = df.query("hour >= @start_hour and hour < @end_hour and rating >= @min_rating")
    vacancies = filtered_df.groupby("id_vacancy")["rating"].mean().sort_values(ascending=False).index.tolist()
    return vacancies


In [ ]:
import pandas as pd
import numpy as np

# Данные с рейтингом в столбце 'rating'
df = pd.read_csv('data.csv')

# Максимальное значение рейтинга
max_rating = df['rating'].max()

# Стандартное отклонение рейтинга
std_rating = df['rating'].std()

# Интервал рейтинга от (max_rating - std_rating) до max_rating
rating_filter = f'rating >= {max_rating - std_rating} and rating <= {max_rating}'

# Фильтр по интервалу часов и рейтинга
hour_filter = 'hour >= 9 and hour <= 17'
filtered_df = df.query(f'{hour_filter} and {rating_filter}')

# Результат
print(filtered_df)


In [ ]:
import pandas as pd

# предположим, что у нас есть два списка рекомендаций - df1 и df2
# каждый из них содержит столбцы 'id вакансии' и 'рейтинг'

# объединяем списки по столбцу 'id вакансии'
merged_df = pd.merge(df1, df2, on='id вакансии', how='outer', suffixes=('_1', '_2'))

# сортируем по столбцу 'рейтинг' в порядке убывания
sorted_df = merged_df.sort_values(by='рейтинг_1', ascending=False)

# фильтруем список так, чтобы в топ-5 попали те вакансии,
# которые есть в обоих списках, сохраняя приоритет за первым списком
top_5 = sorted_df[sorted_df['id вакансии'].isin(df1['id вакансии'])].head(5)


In [ ]:
job_id_to_title = dict(zip(df['job_id'], df['job_title']))

job_title = job_id_to_title[job_id]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# Расчет косинусного расстояния
cosine_sim = cosine_similarity(user_params, job_params)

# Расчет евклидова расстояния
euclidean_dist = euclidean_distances(user_params, job_params)

# Вывод результатов
print("Cosine similarity:\n", cosine_sim)
print("\nEuclidean distance:\n", euclidean_dist)


In [ ]:
import pandas as pd
import numpy as np

# Load data into a DataFrame
data = pd.read_csv('data.csv')

# Convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

# Add columns for hour, day of week, month, and weekday/weekend
data['hour'] = data['timestamp'].dt.hour
data['day_of_week'] = data['timestamp'].dt.day_name()
data['month'] = data['timestamp'].dt.month_name()
data['is_weekend'] = np.where(data['timestamp'].dt.dayofweek < 5, 0, 1)

# Add column for whether user is registered
data['is_registered'] = np.where(data['user_id'].isnull(), 0, 1)

# Group by device ID
grouped_by_device = data.groupby('device_id').agg({
    'hour': lambda x: x.mode()[0],
    'day_of_week': lambda x: x.mode()[0],
    'month': lambda x: x.mode()[0],
    'is_weekend': lambda x: x.mode()[0],
    'rating': 'mean'
})

# Group by vacancy ID
grouped_by_vacancy = data.groupby('vacancy_id').agg({
    'hour': lambda x: x.mode()[0],
    'day_of_week': lambda x: x.mode()[0],
    'month': lambda x: x.mode()[0],
    'is_weekend': lambda x: x.mode()[0],
    'rating': 'mean'
})

# Set the automatic rating range
max_rating = grouped_by_vacancy['rating'].max()
min_rating = grouped_by_vacancy['rating'].max() - np.sqrt(grouped_by_vacancy['rating'].var())

# Filter by time and rating
time_start = '2023-04-01 09:00:00'
time_end = '2023-04-01 17:00:00'
filtered_data = data.query('timestamp >= @time_start & timestamp <= @time_end')
filtered_data = filtered_data.query('@min_rating <= rating <= @max_rating')

# Get the recommended vacancies
recommended_vacancies = filtered_data.groupby('vacancy_id')['rating'].mean().sort_values(ascending=False).index.tolist()
print(recommended_vacancies)


In [ ]:
import pandas as pd

# загрузить данные из таблицы
data = pd.read_table('имя_таблицы.txt', sep='\t')

# группировка по столбцам устройства и вакансии с выбором максимального значения рейтинга
grouped_data = data.groupby(['device_id', 'vacancy_id']).agg({'rating': 'max'})

# объединение с оригинальной таблицей по столбцам устройства, вакансии и рейтингу
result = pd.merge(data, grouped_data, on=['device_id', 'vacancy_id', 'rating'], how='inner')

# сохранить результат в файл
result.to_csv('результат.txt', sep='\t', index=False)


In [ ]:
import pandas as pd

# загрузить данные из таблицы
data = pd.read_table('имя_таблицы.txt', sep='\t')

# группировка по столбцу user_id с выбором максимального значения рейтинга для каждой пары vacancy_id и ip_address
grouped_data = data.groupby(['user_id', 'vacancy_id', 'ip_address']).agg({'rating': 'max'})

# сбросить индексы
grouped_data = grouped_data.reset_index()

# объединение с оригинальной таблицей по столбцам user_id, vacancy_id, ip_address и рейтингу
result = pd.merge(data, grouped_data, on=['user_id', 'vacancy_id', 'ip_address', 'rating'], how='inner')

# сохранить результат в файл
result.to_csv('результат.txt', sep='\t', index=False)


In [ ]:
import pandas as pd

# загрузка данных из файла
df = pd.read_csv('data.csv')

# добавление столбца с часом взаимодействия
df['interaction_hour'] = df['timestamp'].dt.hour

# добавление столбца с днем недели взаимодействия
df['interaction_weekday'] = df['timestamp'].dt.weekday

# добавление столбца с месяцем взаимодействия
df['interaction_month'] = df['timestamp'].dt.month

# добавление столбца, указывающего, является ли день взаимодействия выходным
df['is_weekend'] = df['timestamp'].dt.weekday.isin([5, 6])

# добавление столбца, указывающего, зарегистрирован ли пользователь
df['is_registered'] = df['user_id'].fillna(False).astype(bool)

# сохранение данных в файл
df.to_csv('processed_data.csv', index=False)


In [ ]:
device_grouped = df.groupby('id устройства').agg({
    'час': lambda x: x.mode()[0],
    'день недели': lambda x: x.mode()[0],
    'месяц': lambda x: x.mode()[0],
    'рейтинг': 'mean'
}).reset_index()

job_grouped = df.groupby('id вакансии').agg({
    'час': lambda x: x.mode()[0],
    'день недели': lambda x: x.mode()[0],
    'месяц': lambda x: x.mode()[0],
    'рейтинг': 'mean'
}).reset_index()
